<a href="https://colab.research.google.com/github/troncosofranco/Deep-Learning-Projects/blob/main/PyTorch_w/MLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Import modules

In [5]:
import torch
import torch.nn as nn
from torch.utils import data
import torchvision
import torchvision.datasets
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
!pip install mlflow
import mlflow
import mlflow.pytorch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Print module version.

In [6]:
print("PyTorch: {}".format(torch.__version__))
print("torchvision: {}".format(torchvision.__version__))
print("sklearn: {}".format(sklearn.__version__))
print("MLFlow: {}".format(mlflow.__version__))
print("Numpy: {}".format(np.__version__))
print("Device: ", device)

PyTorch: 1.13.1+cu116
torchvision: 0.14.1+cu116
sklearn: 1.0.2
MLFlow: 2.1.1
Numpy: 1.21.6
Device:  cpu


Define some basic hyperparameters.

In [7]:
batch_size = 256
num_classes = 10
learning_rate = 0.001

#2. Load data

In [8]:
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=None)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=None)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [9]:
x_train, y_train = train_set.data, train_set.targets
x_test, y_test = test_set.data, test_set.targets

Reshape your x-sets.

In [10]:
x_train, y_train = train_set.data, train_set.targets
x_test, y_test = test_set.data, test_set.targets

In [11]:
y_train[0]

tensor(5)

The output is a number, not a vector. Convert the y-sets into a one-hot encoded format.

In [13]:
def to_one_hot(num_classes, labels):
 one_hot = torch.zeros(([labels.shape[0], num_classes]))
 for f in range(len(labels)):
    one_hot[f][labels[f]] = 1
 return one_hot

Convert your y-sets to be in a one-hot encoded format.

In [14]:
y_train = to_one_hot(num_classes, y_train)
y_test = to_one_hot(num_classes, y_test)

In [15]:
y_train[0]

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

 Check the shapes of data sets.

In [ ]:
print("Shapes")
print("x_train: {}\ny_train: {}".format(x_train.shape, y_train.shape))
print("x_test: {}\ny_test: {}".format(x_test.shape, y_test.shape))

#3. MLFlow Run – Training and Evaluating

A popular convention in PyTorch is to define the model as a class since it allows you to much more easily use the GPU while training.

In [17]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        # IN 1x28x28 OUT 16x14x14
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1, dilation=1)
        # IN 16x14x14 OUT 32x6x6
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=0, dilation=1)
        # IN 32x6x6 OUT 64x2x2
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=0, dilation=1)
        # IN 64x2x2 OUT 256
        self.flat1 = nn.Flatten()
        self.dense1 = nn.Linear(in_features=256, out_features=128)
        self.dense2 = nn.Linear(in_features=128, out_features=64)
        self.dense3 = nn.Linear(in_features=64, out_features=10)
 
    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.flat1(x)
        x = self.dense1(x)
        x = nn.ReLU()(x)
        x = self.dense2(x)
        x = nn.ReLU()(x)
        x = self.dense3(x)
        x = nn.Softmax()(x)
        return x

In [18]:
model = model().to(device)
optimizer = torch.optim.Adam(model.parameters(), 
lr=learning_rate)
criterion = nn.BCELoss()

Define a data loader using functionality provided by PyTorch.

In [19]:
dataset = data.TensorDataset(x_train,y_train)
train_loader = data.DataLoader(dataset, batch_size=batch_size)

Define the training loop.

In [23]:
num_epochs = 5
for f in range(num_epochs):
 for batch_num, minibatch in enumerate(train_loader):
    minibatch_x, minibatch_y = minibatch[0], minibatch[1]
    
    output = model.forward(torch.Tensor(minibatch_x.float()).cuda())
    loss = criterion(output, torch.Tensor(minibatch_y.float()).cuda())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch {f} Batch_Num {batch_num} Loss {loss}")

RuntimeError: ignored

Start an MLFlow run.

In [ ]:
mlflow.set_experiment("PyTorch_MNIST")
with mlflow.start_run():
 preds = model.forward(torch.Tensor(x_test.float()).cuda())
 preds = np.round(preds.detach().cpu().numpy())
 eval_acc = accuracy_score(y_test, preds)
 auc_score = roc_auc_score(y_test, preds)
 mlflow.log_param("batch_size", batch_size)
 mlflow.log_param("num_epochs", num_epochs)
 mlflow.log_param("learning_rate", learning_rate)
 mlflow.log_metric("eval_acc", eval_acc)
 mlflow.log_metric("auc_score", auc_score)
 print("eval_acc: ", eval_acc)
 print("auc_score: ", auc_score)
 mlflow.pytorch.log_model(model, "PyTorch_MNIST")
mlflow.end_run()

#4. Load an MLFlow model

In [ ]:
# loaded_model = mlflow.pytorch.load_model("runs:/YOUR_RUN_ID/PyTorch_MNIST")

Obtain predictions and metrics of loaded model.

In [ ]:
# preds = loaded_model.forward(torch.Tensor(x_test.float()).cuda())
# preds = np.round(preds.detach().cpu().numpy())
# eval_acc = accuracy_score(y_test, preds)
# auc_score = roc_auc_score(y_test, preds)
# print("eval_acc: ", eval_acc)
# print("auc_score: ", auc_score)